In [1]:
import tensorflow as tf
import wandb
import dlomix
import dlomix.losses
from dlomix.data import load_processed_dataset
import seaborn as sns
import numpy as np
from matplotlib import pyplot as plt
import yaml
from dlomix.losses import masked_spectral_distance
import tqdm

2024-07-21 15:20:52.140378: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-21 15:20:52.140583: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-21 15:20:52.333440: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-21 15:20:52.872209: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-21 15:21:01.884756: W tensorflow/comp


Avaliable feature extractors are (use the key of the following dict and pass it to features_to_extract in the Dataset Class):
{
   "atom_count": "Atom count of PTM.",
   "delta_mass": "Delta mass of PTM.",
   "mod_gain": "Gain of atoms due to PTM.",
   "mod_loss": "Loss of atoms due to PTM.",
   "red_smiles": "Reduced SMILES representation of PTM."
}.
When writing your own feature extractor, you can either
    (1) use the FeatureExtractor class or
    (2) write a function that can be mapped to the Hugging Face dataset.
In both cases, you can access the parsed sequence information from the dataset using the following keys, which all provide python lists:
    - _parsed_sequence: parsed sequence
    - _n_term_mods: N-terminal modifications
    - _c_term_mods: C-terminal modifications



In [2]:
def load_config(config_path):
    with open(config_path, 'r') as yaml_file:
        config = yaml.safe_load(yaml_file)
        return config

def load_dataset_from_config(config):
    path = config['dataset']['processed_path']
    return load_processed_dataset(path)

In [3]:
noptm_data_config = load_config('../bmpc_shared_scripts/prepare_dataset/config_files/noptm_baseline_small_bs1024_unmod_extended.yaml')
noptm_data = load_dataset_from_config(noptm_data_config)
ptm_data_config = load_config('../bmpc_shared_scripts/prepare_dataset/config_files/ptm_baseline_small_cleaned_bs1024.yaml')
ptm_data = load_dataset_from_config(ptm_data_config)

In [6]:
noptm_data.tensor_train_data.cardinality().numpy()

8

In [11]:
for batch, y_true in tqdm.tqdm(noptm_data.tensor_val_data):
    print(y_true.shape)

  0%|          | 0/4 [00:00<?, ?it/s]

100%|██████████| 4/4 [00:00<00:00, 14.37it/s]

(1024, 174)
(1024, 174)
(1024, 174)
(921, 174)


In [12]:
noptm_data.batch_size

1024